In [1]:
import kagglehub
import pandas as pd
import numpy as np
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

2025-06-09 22:22:44.770816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749496964.818005   53976 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749496964.832359   53976 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749496964.869109   53976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749496964.869146   53976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749496964.869150   53976 computation_placer.cc:177] computation placer alr

In [2]:
# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /home/eugen/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [3]:
dataset = pd.read_csv(path + '/IMDB Dataset.csv')
pd.set_option('display.max_colwidth', None)
dataset.head(3), dataset.info(), dataset.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [4]:
dataset = dataset.sample(n=3000)
dataset['review'] = dataset['review'].apply(lambda row: row.lower())
dataset['review'] = dataset['review'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
dataset.sample(3)

,review,sentiment
5442,im a huge fan of legendary director elia kazan his movies often deal with people trying to overcome their weaknesses be it obsessive love in splendor in the grass poverty in a tree grows in brooklyn or racism in gentlemens agreement while looking for other movies made by kazan i stumbled across a movie that was based on a novel by arthur miller the movie called focus stars william hmacy who happens to be one of my favorite actors and laura dern it deals with antisemitism in a very realistic way br br macys character seems to go along with the bigots in his neighborhood and on his job until it affects him personally thats the key here the reason he acted wasnt because he knew it was wrong and he wanted to take a stand he acted because he was now considered an outsider and he began to experience the same looks the same remarks and the same brutally that mr finklestein david pamyersexperienced br br the movie is very well written and well acted meat loaf does a awesome job of playing a dirt bag both macy and derns performances are outstanding its easier in life to go along with instead of going against sometimes in order for people to take a stand things have to impact their lives or the lives of their loved onesonly then will some find their moral compass,positive
6586,dahl seems to have been under the influence of wenders the american friend innocent nick cage gets recruited for a hit dennis hopper plays a real hit man lara flynn boyle is dangerous the hero gets more entangled the more he tries to extricate hisself and small town america does not seem all that safer than the big city like its predecessor mentioned above this movie has lots of plot twists and turns that seem improbable but all lead to the cathartic self discovery,positive
38935,this was a very good show i enjoyed the construction of real time and flashback seeing the old diggers meeting again and recalling the terrors of their captivity in changi pow camp the main problem with the way the show was written is that the scenes of life in changi are more like a holiday camp than what the place must have been like i am old enough to remember film footage of the men being liberated from changi and other japanese pow camps no actor could lose enough weight to have a resemblance of the state of those men they made the jews of belsen look like sumo wrestlers i have met several veterans from changi over the years many would never ride in a japanese car let alone own one the physical and mental torture those men endured was too horrific for them to even talk about what percentage survived john doyle might be ok writing comedy for roy and hg i hate that too but this is a serious sugar coating of history that should never have been tolerated im happy for satirists to write the life of brian and make fun of the crucifixion because it is obviously comedy even if some consider it to be in bad taste changi is written as a portrayal of a real event and as such might be regarded by younger people as a true record great performances by a fine cast cannot redeem this lightweight screenplay,negative


In [5]:
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')

def tokenize_text(text):
    return tokenizer(
         text.tolist(),
         max_length = 50,
         truncation = True,
         padding = 'max_length',
         return_tensors = 'np'
    )

tokenized = tokenize_text(dataset['review'])
tokenized

{'input_ids': array([[  101,  2023,  1037, ...,  4811,  4081,   102],
       [  101,  2023,  3185, ...,  2237,  2008,   102],
       [  101,  2204, 21416, ...,  2000,  2156,   102],
       ...,
       [  101,  2412,  2144, ...,  3641,  2058,   102],
       [  101,  2469,  2023, ...,  2009,  2003,   102],
       [  101,  2023,  2143, ...,  9755,  2673,   102]], shape=(3000, 50)), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(3000, 50)), 'attention_mask': array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], shape=(3000, 50))}

In [6]:
bert_model = TFBertModel.from_pretrained('prajjwal1/bert-tiny', from_pt=True)
outputs = bert_model(tokenized)

X = outputs.last_hidden_state[:, 0, :].numpy()
y = dataset['sentiment']


I0000 00:00:1749496993.867400   53976 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1749496993.867627   53976 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6260 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.68      0.70      0.69       299
    positive       0.69      0.68      0.68       301

    accuracy                           0.69       600
   macro avg       0.69      0.69      0.69       600
weighted avg       0.69      0.69      0.69       600



In [8]:
import spacy

spacy_nlp = spacy.load("en_core_web_sm")

def extract_actor_sentiment(text):
    doc = spacy_nlp(text)
    actor_sentiment = None
    for token in doc:
        if token.lemma_ == "actor" or token.lemma_ == "actors" and token.pos_ == "NOUN":
            for child in token.children:
                if child.pos_ == "ADJ":
                    sentiment = child.text.lower()
                    if sentiment in ['good', 'great', 'excellent', 'amazing', 'brilliant', 'fantastic']:
                        actor_sentiment = 0
                    elif sentiment in ['bad', 'terrible', 'awful', 'poor', 'horrible']:
                        actor_sentiment = 1
                    break
            if actor_sentiment:
                break
    return actor_sentiment


dataset['actor_sentiment'] = dataset['review'].apply(extract_actor_sentiment)



In [9]:
dataset_with_actors_sent = dataset[dataset['actor_sentiment'].notna()]
dataset_with_actors_sent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 47744 to 17047
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review           66 non-null     object 
 1   sentiment        66 non-null     object 
 2   actor_sentiment  66 non-null     float64
dtypes: float64(1), object(2)
memory usage: 2.1+ KB


In [10]:
X_by_actor = dataset_with_actors_sent.drop('actor_sentiment', axis=1)
y_by_actor = dataset_with_actors_sent['actor_sentiment']

tokenized = tokenize_text(X_by_actor['review'])
outputs_by_actor = bert_model(tokenized)
X_by_actor = outputs_by_actor.last_hidden_state[:, 0, :].numpy()

X_train_actor, X_test_actor, y_train_actor, y_test_actor = train_test_split(X_by_actor, y_by_actor, test_size = 0.2)

log_reg_by_actor = LogisticRegression(max_iter=1000)
log_reg_by_actor.fit(X_train_actor, y_train_actor)
y_pred_actor = log_reg_by_actor.predict(X_test_actor)
print(classification_report(y_test_actor, y_pred_actor))

              precision    recall  f1-score   support

         0.0       0.77      0.91      0.83        11
         1.0       0.00      0.00      0.00         3

    accuracy                           0.71        14
   macro avg       0.38      0.45      0.42        14
weighted avg       0.60      0.71      0.65        14

